In [1]:
# Import dependencies.
import pandas as pd
import numpy as np
from citipy import citipy
# Import the API key.
from config import weather_api_key
# Import the requests library.
import requests

## Imports that we used in the modules, but were not needed for this deliverable.
# import matplotlib.pyplot as plt
# Import the time module.
# import time
# Import the datetime module from the datetime library.
# from datetime import datetime
# Import linear regression from the SciPy stats module.
# from scipy.stats import linregress

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [8]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

773

In [5]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

#loop through cities in list
for i, city in enumerate(cities):
    
    #group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    #create URL endpoint for city
    city_url = url + '&q=' + city.replace(" ","+")
    
    #log the URL, record, and set numbers and the city
    print(f'Processing Record {record_count} of Set {set_count} | {city}')
    # add 1 to record count
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_country = city_weather["sys"]["country"]
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_weather_dc = city_weather["weather"][0]["description"]
        # Convert the date to ISO standard.
        # city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_weather_dc})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | tiksi
Processing Record 2 of Set 1 | wanning
Processing Record 3 of Set 1 | avera
Processing Record 4 of Set 1 | busselton
Processing Record 5 of Set 1 | souillac
Processing Record 6 of Set 1 | sao filipe
Processing Record 7 of Set 1 | bredasdorp
Processing Record 8 of Set 1 | ilulissat
Processing Record 9 of Set 1 | buqayq
City not found. Skipping...
Processing Record 10 of Set 1 | boyolangu
Processing Record 11 of Set 1 | hermanus
Processing Record 12 of Set 1 | new norfolk
Processing Record 13 of Set 1 | albany
Processing Record 14 of Set 1 | hithadhoo
Processing Record 15 of Set 1 | illoqqortoormiut
City not found. Skipping...
Processing Record 16 of Set 1 | meulaboh
Processing Record 17 of Set 1 | caravelas
Processing Record 18 of Set 1 | castro
Processing Record 19 of Set 1 | cherskiy
Processing Record 20 of Set 1 | port alfred
Processing Record 21 of Set 1 | punta arenas
Processing Record 

In [6]:
# Recall that we can convert a list of dictionaries to a Pandas DataFrame using df = pd.DataFrame(list with dictionaries).
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Tiksi,RU,71.6872,128.8694,-8.73,99,57,4.27,broken clouds
1,Wanning,CN,18.8003,110.3967,61.59,88,100,9.60,overcast clouds
2,Avera,US,33.1940,-82.5271,42.75,26,0,12.95,clear sky
3,Busselton,AU,-33.6500,115.3333,67.05,91,7,13.53,clear sky
4,Souillac,MU,-20.5167,57.5167,79.18,83,20,5.75,few clouds
5,Sao Filipe,CV,14.8961,-24.4956,71.10,72,100,13.13,overcast clouds
6,Bredasdorp,ZA,-34.5322,20.0403,66.27,77,5,6.91,clear sky
7,Ilulissat,GL,69.2167,-51.1000,5.58,69,100,14.85,overcast clouds
8,Boyolangu,ID,-8.1181,111.8935,76.24,90,100,1.70,overcast clouds
9,Hermanus,ZA,-34.4187,19.2345,66.79,77,2,13.80,clear sky


In [7]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")